# 1. Contextualizado o Problema

## 1.1 Contexto Geral

- O CEO da Amazon contratou um time de cientistas de dados para trabalhar em diferentes vertentes da empresa, e você ficou encarregado de fazer um sistema de recomendação para os clientes Amazon. Para isso, te deram acesso à uma base de dados não muito estruturada em Json: um arquivo de metadata com informações dos produtos e outro com as avaliações.

- O CEO deseja que seu algoritmo de recomendação seja exclusivamente em cima de avaliações verificadas (campo `verified`=True no arquivo de avaliações). Contudo, há uma base sem classificação e que o CEO faz questão de que seja
adicionada no sistema de recomendação (valores com missing value na coluna `verified`). 

- Para isso, você precisará classificar se estas avaliações são verificadas ou não, e no caso positivo, adicioná-los no sistema de recomendação.

- Adicionalmente, o CEO tam bém deseja saber de possíveis associações e/ou correlações nesta base de dados que você vai trabalhar.

## 1.2 Objetivos

- Carregar, limpar e fazer análises exploratórias no banco de dados fornecido;

- Classificar a base sem informação para incluir as observações das avaliações verificadas no sistema de recomendação;

- Desenvolver e entregar um sistema de recomendação, com exemplos de aplicação.

- O desenvolvimento e a decisão do modelo é totalmente sua, portanto se achar que deve utilizar um valor/ procedimento diferente, sinta-se livre para fazer os testes e validar suas hipóteses para achar o resultado coerente.

- LIMPEZA DOS DADOS: Aqui você deve limpar e explorar os dados para decidir variáveis que possam ser úteis para classificação de avaliações verificadas. Você também deve levantar associações e/ou correlação para apresentar ao CEO


- EDA Realize uma EDA da forma como preferir, explore os dados, levante ideias, avalie correlações.

  - 1° ETAPA CLASSIFICAÇÃO: Aqui você deve utilizar as variáveis estudadas na primeira parte para classificar as avaliações sem label. Deve-se avaliar os modelos criados, tunar ao menos um modelo selecionado e utilizá-lo na classificação. Após classificação, não se esqueça de remover as avaliações
não verificadas.

  - 2° ETAPA RECOMENDAÇÃO: Aqui você deve desenvolver um sistema de recomendação e mostrá-lo na prática para o CEO. Tente prever possíveis erros
para evitá-los de antemão. Por exemplo, caso o produto requisitado não esteja na matriz/base, aponte os 10 mais vendidos.

# 2. Pacotes Python

In [ ]:
# Bibliotecas Gerais
import numpy                   as np
import pandas                  as pd
import matplotlib.pyplot       as plt
import seaborn                 as sns
import sys
import time
from scipy                     import stats

# Bibliotecas do Colab
from google.colab              import drive

# Bibliotecas Classificação
from sklearn                       import metrics
from sklearn.preprocessing         import MinMaxScaler
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.naive_bayes           import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.svm                   import SVC
from sklearn.neural_network        import MLPClassifier
from sklearn.metrics               import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics               import classification_report
from sklearn.metrics               import plot_confusion_matrix
from sklearn.metrics               import plot_roc_curve

# Bibliotecas de Recomendação
from sklearn.metrics.pairwise      import cosine_similarity

# Redução de Dimensionalidade
from sklearn.decomposition         import PCA
from sklearn.decomposition         import TruncatedSVD

# 3. Leitura de Datasets

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
pip freeze --local > /content/gdrive/MyDrive/Colab Notebooks/Super Chellenge/requeriments.txt

In [ ]:
link  = '/content/gdrive/MyDrive/Colab Notebooks/Slot7-Assignment/01QualidadeMatPrima.csv'
df     = pd.read_csv(link,sep=";",decimal=',')
df1_save = df1.copy()

# 4. EDA

In [ ]:
df.head(5)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
pd.DataFrame(df.isnull().sum(axis=0)).sort_values(by=0, ascending=False)/df.shape[0]

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(df.corr(), annot=True,cmap='BrBG')
plt.show()
plt.close()

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
colunas = df.select_dtypes(include='number').columns
ncolunas = len(colunas)

for i in range(0,ncolunas):
  plt.figure(figsize=(20,20))
  sns.displot(x=colunas[i],data=df)
  plt.show()
  plt.close()

# 5. Classificador  

In [ ]:
X = df.drop(columns=["target"])
Y = df["target"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test: {X_test.shape}")
print(f"Shape y_test: {y_test.shape}")

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_test = pd.DataFrame(X_test_scaled,columns = X_test.columns)

In [ ]:
scores_list = []
K_neighbors = range(1,20)

for k in K_neighbors:

  knn =  KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  scores_list.append(accuracy_score(y_test, y_pred))

In [ ]:
plt.plot(K_neighbors, scores_list)
plt.xlabel("Valor de K")
plt.ylabel("Acurácia")

In [ ]:
models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'LogisticRegression': LogisticRegression(),
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion="entropy"),
    'RandomForestClassifier': RandomForestClassifier(criterion='entropy', n_estimators=150),
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
    'BernoulliNB': BernoulliNB(),
    'SVC_Poly': SVC(kernel='poly'),
    'SVC_Linear': SVC(kernel='linear'),
    'KNeighborsClassifier': KNeighborsClassifier(n_neighbors=5),
    'MLPClassifier': MLPClassifier(hidden_layer_sizes=(100, 50, 20),activation='relu'),
 }

In [ ]:
f1score_list    = []
clf_list        = []

for name, model in models.items():

  clf = model
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)

  f1score   = np.mean(f1_score(y_test, y_pred,average=None))*100
  f1score_list.append(f1score)
  
  clf_list.append(name)
  
  print(name)
  print(f"F1: {f1score}%")
  print(classification_report(y_test, y_pred))

  plot_confusion_matrix(clf, X_test, y_test, display_labels=list_classes, values_format='d')
  plt.grid(False)
  plt.show()

In [ ]:
df_metricas = pd.DataFrame({'Classificador': clf_list, 'F1-Score': f1score_list})

In [ ]:
df_metricas.sort_values(by='F1-Score', ascending=False)